<a href="https://colab.research.google.com/github/subodh2708/deep-learning---tensorflow-2.0/blob/main/08_NLP_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP in Tensorflow

In [1]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2022-09-18 11:16:08--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-18 11:16:08 (75.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
unzip_data('nlp_getting_started.zip')


--2022-09-18 11:16:11--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.128, 142.250.31.128, 142.251.111.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-09-18 11:16:11 (130 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [4]:
 import pandas as pd
 train_df = pd.read_csv("train.csv")
 test_df = pd.read_csv("test.csv")
 train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df),len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[['text','target']][random_index:random_index+5].itertuples():
  _,text,target = row
  print(f"target:{target}","(real disaster)" if target > 0 else  "(not a disaster)")
  print(f"Text\n {text} \n")
  print("---\n")

target:1 (real disaster)
Text
 #breakingnews Rly tragedy in MP: Some live to recount horror: ÛÏWhen I saw coaches of my train plunging into wa... http://t.co/eYOrBmF3O3 

---

target:0 (not a disaster)
Text
 New Expansion Ideas - Bard Class Holy Trinity + 1 http://t.co/EGioxBabOe 

---

target:1 (real disaster)
Text
 Wtf this mom just drowned her child?! 

---

target:1 (real disaster)
Text
 Route Complex AM Fire Update (Route Complex Wildfire): FIRE UPDATE: ROUTE COMPLEX Thursday Morning Aug... http://t.co/nS5lBS5ZUp #CAFire 

---

target:0 (not a disaster)
Text
 The #tubestrike is because TFL workers may have trouble planning downtime. I hope none need emergency services. http://t.co/iCSFDSiFqb 

---



## splitting the data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_sentences, val_sentences,train_labels,val_labels = train_test_split(train_df_shuffled['text'].to_numpy(),
                                                                          train_df_shuffled['target'].to_numpy(),
                                                                          test_size = 0.1,
                                                                          random_state = 42)

## converting text into numbers

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split = 'whitespace',
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length = None,
                                    pad_to_max_tokens= False
                                    )

In [13]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [14]:
max_vocab_length = 10000
max_length = 15

In [15]:
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode = 'int',
                                    output_sequence_length = max_length)

In [16]:
text_vectorizer.adapt(train_sentences)

In [17]:
sample_sentence = "there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [18]:
random_sentence = random.choice(train_sentences)
print(f'Original text:\n {random_sentence} \
      \n Vectorized version:\n {text_vectorizer([random_sentence])}')

Original text:
 @Deosl86 @xavier_marquis Hostages are meaningless might as well just play cod search and destroy.       
 Vectorized version:
 [[   1 6613  422   22    1  843   26  251   29  673 5960 1259    7  305
     0]]


In [19]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"number of words in vocab:{len(words_in_vocab)}")
print(f"top 5 word in vocab:{top_5_words}")
print(f"bottom 5 words in vocab:{bottom_5_words}")

number of words in vocab:10000
top 5 word in vocab:['', '[UNK]', 'the', 'a', 'in']
bottom 5 words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


## Creating the Embedding using Embedding layer

In [20]:
from tensorflow.keras.layers import Embedding

embedding = Embedding(input_dim = max_vocab_length,
                      output_dim = 128,
                      embeddings_initializer ='uniform',
                      input_length = max_length)
embedding

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence} \
        \n Embedded Version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 I liked a @YouTube video http://t.co/ki1yKrs9fi Minecraft: NIGHT LUCKY BLOCK MOD (BOB APOCALYPSE WITHER 2.0 &amp; MORE!) Mod Showcase         
 Embedded Version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.00835628,  0.02166018,  0.01478944, ...,  0.02710161,
         -0.01041956,  0.04809072],
        [ 0.02464001,  0.04199305, -0.00515845, ..., -0.02179166,
         -0.02821803, -0.04968426],
        [-0.01830717,  0.0431429 ,  0.00464444, ...,  0.00713617,
         -0.01294733,  0.02858562],
        ...,
        [ 0.02891712,  0.0004584 , -0.04222882, ..., -0.0235206 ,
         -0.02144824, -0.02181759],
        [-0.04471926,  0.03795146, -0.00575781, ...,  0.0126325 ,
          0.02470387,  0.01762786],
        [ 0.02678714, -0.02348839, -0.04447775, ...,  0.00788601,
          0.03833959, -0.03631723]]], dtype=float32)>

In [22]:
random_sentence ,sample_embed[0][0],sample_embed[0][0].shape

('I liked a @YouTube video http://t.co/ki1yKrs9fi Minecraft: NIGHT LUCKY BLOCK MOD (BOB APOCALYPSE WITHER 2.0 &amp; MORE!) Mod Showcase',
 <tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.00835628,  0.02166018,  0.01478944, -0.04849362,  0.04813394,
         0.02922629,  0.00379537, -0.03869253, -0.00934242,  0.04087781,
         0.02525032,  0.04749559,  0.00054858,  0.00507587,  0.02023808,
        -0.01632845, -0.0021466 , -0.02605766, -0.01698022, -0.0301997 ,
        -0.00975994,  0.02371189, -0.00765684,  0.03100291,  0.04063005,
        -0.03958132,  0.04296739,  0.02969852,  0.01693454,  0.00646168,
        -0.0007875 ,  0.02433098, -0.00649951, -0.01999271, -0.02696161,
        -0.04867189,  0.04090729,  0.01490904, -0.02307434,  0.04282198,
         0.00740212,  0.03329524,  0.03585145, -0.01633392,  0.03294679,
        -0.03931642, -0.01140318,  0.03246126, -0.03490448, -0.00098982,
        -0.01339085, -0.02871783, -0.03827624, -0.03350504, -0.02248404,
         0

## Modelling text Dataset 

## Model 0: Getting the baseline model

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f' baseline accuracy : {baseline_score *100:.2f}%')

 baseline accuracy : 79.27%


In [25]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## creating our evaluation function for our model experiments

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_result(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = 'weighted')
  model_results = {'accuracy':model_accuracy,
                   'precision':model_precision,
                   'recall':model_recall,
                   'f1':model_f1}

  return model_results


In [27]:
baseline_results = calculate_result(y_true=val_labels,
                                    y_pred = baseline_preds)
print(baseline_results)

{'accuracy': 0.7926509186351706, 'precision': 0.8111390004213173, 'recall': 0.7926509186351706, 'f1': 0.7862189758049549}


##Model 1: simple dense model

In [28]:
from helper_functions import create_tensorboard_callback

In [29]:
SAVE_DIR = 'model_logs'

In [30]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype= tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_1 = tf.keras.Model(inputs,outputs, name= 'model_1_dense')

In [31]:
model_1.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [32]:
model_1_history= model_1.fit(x = train_sentences,
                           y= train_labels,
                           epochs = 5,
                           validation_data = (val_sentences,val_labels),
                           callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                    experiment_name = 'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20220918-111614
Epoch 1/5
215/215 [==============================] - 7s 24ms/step - loss: 0.6133 - accuracy: 0.6907 - val_loss: 0.5359 - val_accuracy: 0.7520
Epoch 2/5
215/215 [==============================] - 4s 21ms/step - loss: 0.4440 - accuracy: 0.8183 - val_loss: 0.4696 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 5s 22ms/step - loss: 0.3490 - accuracy: 0.8587 - val_loss: 0.4600 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 4s 20ms/step - loss: 0.2863 - accuracy: 0.8886 - val_loss: 0.4642 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2389 - accuracy: 0.9118 - val_loss: 0.4875 - val_accuracy: 0.7940


In [33]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4875 - accuracy: 0.7940


[0.4875306189060211, 0.7939632534980774]

In [34]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0:10]

array([[0.28469405],
       [0.78955376],
       [0.99771565],
       [0.10372725],
       [0.10467237],
       [0.93154705],
       [0.900244  ],
       [0.9931536 ],
       [0.9543351 ],
       [0.24036735]], dtype=float32)

In [35]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [36]:
model_1_results = calculate_result(y_true = val_labels,
                                   y_pred = model_1_preds)
model_1_results

{'accuracy': 0.7939632545931758,
 'precision': 0.8019542022432935,
 'recall': 0.7939632545931758,
 'f1': 0.7902387415131078}

In [37]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


array([ True, False,  True,  True])

## Visualising learned embeddings

In [38]:
word_in_vocab = text_vectorizer.get_vocabulary()
len(word_in_vocab) , words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [39]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [40]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape

(10000, 128)

In [41]:
import io

# Code to save trained embeddings to file - we got this from here: https://www.tensorflow.org/tutorials/text/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [42]:
# Let's download the saved embeddings locally
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks

###Model 2 : LSTM

In [43]:
# Create LSTM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
# x = layers.LSTM(64, activation="tanh", return_sequences=True)(x) # use return_sequences=True if you want to stack recurrent layers 
# print(f"After LSTM cell with return_sequences=True: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
print(f"After LSTM cell: {x.shape}")
# x = layers.Dense(64, activation="relu")(x) # optional dense layer to have on top of LSTM layer
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cell: (None, 64)


In [44]:
model_2.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [45]:
history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs= 5,
                        validation_data = (val_sentences, val_labels),
                        callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                 experiment_name = "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20220918-111638
Epoch 1/5
215/215 [==============================] - 9s 34ms/step - loss: 0.2198 - accuracy: 0.9191 - val_loss: 0.5456 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.1624 - accuracy: 0.9406 - val_loss: 0.6923 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 6s 29ms/step - loss: 0.1294 - accuracy: 0.9505 - val_loss: 0.6560 - val_accuracy: 0.7861
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.1054 - accuracy: 0.9615 - val_loss: 0.9228 - val_accuracy: 0.7808
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0883 - accuracy: 0.9656 - val_loss: 0.9205 - val_accuracy: 0.7822


In [46]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

((762, 1), array([[2.1827370e-02],
        [7.3310888e-01],
        [9.9977219e-01],
        [7.4510574e-03],
        [5.1501393e-04],
        [9.9921393e-01],
        [9.0231860e-01],
        [9.9985081e-01],
        [9.9975425e-01],
        [1.9732812e-01]], dtype=float32))

In [47]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [48]:
model_2_results = calculate_result(y_true= val_labels,
                                   y_pred = model_2_preds)
model_2_results

{'accuracy': 0.7821522309711286,
 'precision': 0.7868451603977311,
 'recall': 0.7821522309711286,
 'f1': 0.779088324447517}

In [49]:
baseline_results

{'accuracy': 0.7926509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 3: GRU

In [50]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype= 'string')
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
x = layers.Dense(64, activation= 'relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs, name='model_3_GRU')

In [51]:
model_3.compile(loss = 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [52]:
history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs =5,
                        validation_data =(val_sentences, val_labels),
                        callbacks= [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                experiment_name='model_3_GRU')])

Saving TensorBoard log files to: model_logs/model_3_GRU/20220918-111722
Epoch 1/5
215/215 [==============================] - 7s 25ms/step - loss: 0.1552 - accuracy: 0.9403 - val_loss: 0.8580 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0806 - accuracy: 0.9724 - val_loss: 0.8838 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0704 - accuracy: 0.9724 - val_loss: 0.9153 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0602 - accuracy: 0.9740 - val_loss: 1.1654 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0506 - accuracy: 0.9774 - val_loss: 1.4106 - val_accuracy: 0.7612


In [53]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[6.7748427e-03],
       [7.7515411e-01],
       [9.9997616e-01],
       [8.0796331e-02],
       [1.3576781e-05],
       [9.9997461e-01],
       [9.9432957e-01],
       [9.9999297e-01],
       [9.9998641e-01],
       [9.6370816e-01]], dtype=float32)

In [54]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [55]:
model_3_results = calculate_result(y_true= val_labels,
                                   y_pred = model_3_preds)
model_3_results

{'accuracy': 0.7611548556430446,
 'precision': 0.7627664723727715,
 'recall': 0.7611548556430446,
 'f1': 0.7588524417705016}

### Model 4: Bidirectional RNN

In [58]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype='string')
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences= True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs,outputs,name='model_4_bidirectional')

In [59]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [60]:
model_4.compile(loss= 'binary_crossentropy',
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [62]:
history_4 = model_4.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data = (val_sentences,val_labels),
                        callbacks = [create_tensorboard_callback(dir_name= SAVE_DIR,
                                                                 experiment_name= 'model_4_bidirectional')])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20220918-112603
Epoch 1/5
215/215 [==============================] - 16s 54ms/step - loss: 0.1058 - accuracy: 0.9707 - val_loss: 1.0393 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 11s 52ms/step - loss: 0.0520 - accuracy: 0.9772 - val_loss: 1.3375 - val_accuracy: 0.7480
Epoch 3/5
215/215 [==============================] - 10s 45ms/step - loss: 0.0482 - accuracy: 0.9778 - val_loss: 1.2483 - val_accuracy: 0.7690
Epoch 4/5
215/215 [==============================] - 9s 43ms/step - loss: 0.0432 - accuracy: 0.9807 - val_loss: 1.1405 - val_accuracy: 0.7625
Epoch 5/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0395 - accuracy: 0.9809 - val_loss: 1.3473 - val_accuracy: 0.7638


In [63]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[6.5348744e-03],
       [6.1759400e-01],
       [9.9998820e-01],
       [3.6429518e-01],
       [3.8969254e-05],
       [9.9993718e-01],
       [8.8991892e-01],
       [9.9999630e-01],
       [9.9998856e-01],
       [9.9636853e-01]], dtype=float32)

In [64]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [66]:
model_4_results = calculate_result(y_true= val_labels,
                                   y_pred = model_4_preds)
model_4_results

{'accuracy': 0.7637795275590551,
 'precision': 0.765729922717656,
 'recall': 0.7637795275590551,
 'f1': 0.7613639638656104}